In [1]:
import os
import torch
import torchvision
from torchvision.transforms import transforms
from torchvision import datasets
import torch.functional as F

In [2]:
training_transforms = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

In [3]:
# Download and load the dataset
train_dataset = datasets.MNIST(root = './data', train = True, download = True, transform = training_transforms)

In [4]:
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = 128,
    shuffle = True
)

In [5]:
for img, _ in train_loader:
    print(img.size())
    break

torch.Size([128, 1, 28, 28])


In [6]:
def get_mean_std(loader):

    mean = 0
    std = 0
    total_image_count = 0

    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, 28*28)
        mean += images.mean(1).sum(0)
        std += images.std(1).sum(0)
        total_image_count += image_count_in_a_batch

    mean /= total_image_count
    std /= total_image_count

    return mean, std

In [7]:
get_mean_std(train_loader)

(tensor(0.1307), tensor(0.3015))

In [8]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np

In [9]:
mean = [0.1307]
std = [0.3015]

train_transforms = transforms.Compose([
    # Reshaping all the images to 28 X 28 size
    transforms.Resize((28, 28)),
    transforms.RandomHorizontalFlip(), # If we are not passing any values,
                                       # It will by default Flip the images with 50% prob.
    transforms.RandomRotation(10), # Random Rotation to 10 degrees
    # Changing the data type from numpy array to tensor
    transforms.ToTensor(),
    # Normalizing the images using the mean and std that we found in the previous part
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

# In the test dataset we do not need to do augmentation
test_transforms = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [10]:
# Download and load the dataset
train_dataset = datasets.MNIST(root = './data', train = True, transform = training_transforms)
test_dataset = datasets.MNIST(root = './data', train = False, transform = test_transforms)

In [11]:
# Making a custom Loader
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = 64,
    shuffle = True
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle = False
)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


Neural Network

In [13]:
import torch.nn as nn

In [14]:
class MnistModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, images):
        batch_size = images.size(0)
        images = images.reshape(batch_size, -1)
        out = self.linear(images)
        return out

In [15]:
input_size = 28*28
num_classes = 10

In [16]:
model = MnistModel(input_size, num_classes)

In [17]:
model = model.to(device)

In [18]:
loss_fn = nn.CrossEntropyLoss()

In [19]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr = 1e-3,
    momentum = 0.9,
    weight_decay = 0.03
)

In [20]:
def evaluate_model_test(model, test_data_loader):

    model.eval()

    predicted_correctly_on_epoch = 0
    total = 0

    with torch.no_grad():
        for data in test_data_loader:
            images, labels = data

            images = images.to(device)
            labels = labels.to(device)

            total += labels.size(0)

            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)

            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100.0 * predicted_correctly_on_epoch / total

    print("\t\t- Testing dataset. Got %d out of %d images correctly (%.3f%%)" % (predicted_correctly_on_epoch, total, epoch_acc))

    return epoch_acc

In [21]:
def save_checkpoint(model, epoch, optimizer, best_acc):
    state = {
        "epoch": epoch+1,
        "model": model.state_dict(),
        "best accuracy": best_acc,
        "optimizer": optimizer
    }

    torch.save(state, "mnist-logistic-2.pth.tar")

In [24]:
def train_nn(model, train_data_loader, test_data_loader, criterion, optimizer, n_epochs):

    for epoch in range(n_epochs):
        print("Epoch Number %d" %(epoch+1))
        
        model.train()

        running_loss = 0
        running_correct = 0
        total = 0
        best_acc = 0

        for data in train_data_loader:

            images, labels = data

            images = images.to(device)
            labels = labels.to(device)

            total += labels.size(0)

            optimizer.zero_grad()

            outputs = model(images)

            _, pred = torch.max(outputs.data, 1)

            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()

            running_correct += (labels == pred).sum().item()
        
        epoch_loss = running_loss / len(train_loader)

        epoch_acc = 100 * running_correct / total

        print("\t\t- Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f"
              % (running_correct, total, epoch_acc, epoch_loss))
        
        test_dataset_acc = evaluate_model_test(model, test_data_loader)

        if (test_dataset_acc > best_acc):
            best_acc = test_dataset_acc
            save_checkpoint(model, epoch, optimizer, best_acc)

    # Print a message when the training ia completed
    print("Finished")

    # Return the training model
    return model

In [25]:
model = train_nn(model = model,
                 train_data_loader = train_data_loader,
                 test_data_loader = test_data_loader,
                 criterion = loss_fn,
                 optimizer = optimizer,
                 n_epochs = 10)

Epoch Number 1
		- Training dataset. Got 51801 out of 60000 images correctly (86.335%). Epoch loss: 1.250
		- Testing dataset. Got 8624 out of 10000 images correctly (86.240%)
Epoch Number 2
		- Training dataset. Got 52444 out of 60000 images correctly (87.407%). Epoch loss: 1.141
		- Testing dataset. Got 8676 out of 10000 images correctly (86.760%)
Epoch Number 3
		- Training dataset. Got 52706 out of 60000 images correctly (87.843%). Epoch loss: 1.102
		- Testing dataset. Got 8695 out of 10000 images correctly (86.950%)
Epoch Number 4
		- Training dataset. Got 52843 out of 60000 images correctly (88.072%). Epoch loss: 1.085
		- Testing dataset. Got 8703 out of 10000 images correctly (87.030%)
Epoch Number 5
		- Training dataset. Got 52938 out of 60000 images correctly (88.230%). Epoch loss: 1.076
		- Testing dataset. Got 8728 out of 10000 images correctly (87.280%)
Epoch Number 6
		- Training dataset. Got 53000 out of 60000 images correctly (88.333%). Epoch loss: 1.071
		- Testing da